In [1]:
import os,sys,glob
smodelsPath = os.path.expanduser('~/smodels')
sys.path.append(smodelsPath)
from smodels.theory.auxiliaryFunctions import rescaleWidth, unscaleWidth
import numpy as np
from trimRegularGrid import trim1D, testTrimmedGrid, removeWidth

### Trim data along width dimension

In [2]:
maxDiff=5e-2
minValue=1e-8
datasetNames = ['c000','c100','c200','c300']
# txNames = ['THSCPM1b','THSCPM2b','THSCPM3','THSCPM4','THSCPM5', 'THSCPM6','THSCPM7','THSCPM8','THSCPM9']
# txNames = ['THSCPM5']
txNames = ['THSCPM1b','THSCPM2b']
zlabels = ['width']
filenames = []
for tx in txNames:
    for ds in datasetNames:
        dataFile = '%s_efficiencyMaps_%s.dat' %(tx,ds)
        if not os.path.isfile(dataFile):
            continue
        filenames.append(dataFile)


for zlabel in zlabels:
    for i,dataFile in enumerate(filenames[:]):
        print('\n',dataFile)
        #Get column name for efficiency
        valueLabel = [v for v in os.path.splitext(dataFile)[0].split('_') if v[0] == 'c'][0]
        #Get data
        data = np.genfromtxt(dataFile,names=True)
        if not zlabel in data.dtype.names:
            continue
        
        xlabels = [x for x in data.dtype.names if not 'ctau' in x.lower() and not x == valueLabel and not x == zlabel]
        xpts = [list(x) for x in data[xlabels]]
        ypts = data[valueLabel]
        zpts = [rescaleWidth(z) for z in data[zlabel]]
        data[zlabel] = np.array(zpts)
        keepLines = trim1D(zpts,xpts,ypts,maxDiff=maxDiff,minY=minValue)

        #Save trimmed grid to file:
        data = np.genfromtxt(dataFile,names=True)  #Get original data
        newF = os.path.splitext(dataFile)[0]+'_trimmed%s.dat'%(zlabel)
        header = '%19s'*len(data.dtype.names) %data.dtype.names
        header = header[3:]
        newData = data[keepLines]
        np.savetxt(newF,newData,header=header,fmt = ['     %1.7e']*len(data.dtype.names))
        
        filenames[i] = newF #Update file for trimming



 THSCPM1b_efficiencyMaps_c000.dat
Data successfully trimmed down to 40% (642/1600) in 0.0 min

 THSCPM1b_efficiencyMaps_c100.dat
Data successfully trimmed down to 36% (569/1600) in 0.0 min

 THSCPM1b_efficiencyMaps_c200.dat
Data successfully trimmed down to 31% (492/1600) in 0.0 min

 THSCPM1b_efficiencyMaps_c300.dat
Data successfully trimmed down to 26% (415/1600) in 0.0 min


### Remove width data from maps:

In [3]:
datasetNames = ['c000','c100','c200','c300']

for tx in txNames:
    for ds in datasetNames:
        dataFile = '%s_efficiencyMaps_%s.dat' %(tx,ds)
        if not os.path.isfile(dataFile):
            continue        
        print('\n',dataFile)
        #Get data
        data = np.genfromtxt(dataFile,names=True)
        #Remove width
        reducedData = removeWidth(data,label='width')
    
    
        newF = os.path.splitext(dataFile)[0]+'_noWidth.dat'
        header = '%19s'*len(data.dtype.names) %data.dtype.names
        header = header[3:]
        np.savetxt(newF,reducedData,header=header,fmt = ['     %1.7e']*len(data.dtype.names))



 THSCPM1b_efficiencyMaps_c000.dat
Removing widths reduced data down to 3% (50/1600)

 THSCPM1b_efficiencyMaps_c100.dat
Removing widths reduced data down to 3% (50/1600)

 THSCPM1b_efficiencyMaps_c200.dat
Removing widths reduced data down to 3% (50/1600)

 THSCPM1b_efficiencyMaps_c300.dat
Removing widths reduced data down to 3% (50/1600)
